# Pricing Spread Options with Python

##About this notebook
Documention prepared by **Jesus Perez Colino**.
Version 0.1, Released 25/11/2014, Alpha.


- This work is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). This work is offered for free, with the hope that it will be useful.


- **Summary**: This technical note has as main goal to summarize the different methodologies to calculate an spread option, with the implementation in Python. 


- **Index**:

    0. About this notebook
    1. Spreads Options: General Framework
    2. Spread Options Valuation when $K=0$: The Margrabe Formula
    3. Spread Options Valuation when $K \neq 0$: The Kirk Formula
    4. Spread Options Valuation by Monte-Carlo Simulation


- **Reference**: [Rene Carmona and Valdo Durrleman "*Pricing and Hedging Spread Options*" SIAM Review Vol.45 No.4, pp627-685](http://www.princeton.edu/~rcarmona/download/fe/sirev.pdf)


- **Reproducibility conditions**: the content in this technical note can be reproduce in your computer under the following versions of the Python's packages: 


In [93]:
%matplotlib inline
import sys
import numpy as np
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from scipy.stats import norm
import scipy.stats as st

print ' Reproducibility conditions for the Two-factor model'.center(80, '-')
print ' '
print ' Python Version: ' + sys.version
print ' Matplotlib Version:' + matplotlib.__version__
print ' Numpy Version: ' + np.__version__
print ' ' 
print '-'*80

-------------- Reproducibility conditions for the Two-factor model--------------
 
 Python Version: 2.7.10 |Anaconda 2.3.0 (x86_64)| (default, May 28 2015, 17:04:42) 
[GCC 4.2.1 (Apple Inc. build 5577)]
 Matplotlib Version:1.4.3
 Numpy Version: 1.9.2
 
--------------------------------------------------------------------------------


#Spread Options: General Framework

All over this technical note, we will consider a set of Brownian motions $B_{1}(t),B_{2}(t)$ with correlation $\rho$ on the probability space $(\Omega,\mathcal{F},\mathbb{P} )$ endowed with the natural (completed and right-continuous) filtration $\mathbb{F}=(\mathcal{F_t})_{t \geq T}$ generated by ${B_{1}(t),B_{2}(t)}$ up to some fixed time horizon $T>0$. 

We look at the classical setting where besides a riskless bank account with constant interest rate r, our arbitrage-free market model comprises two assets whose prices at time $t$ are denoted by $S_1(t)$ and $S_2(t)$. We assume that their risk-neutral price dynamics are given by the following stochastic differential equations:


\begin{equation}
\begin{array}{lll}
\frac{dS_1(t)}{S_1(t)}& =& \mu_1 (t,T) dt+\sigma_{1}(t,T)dB_{1}(t)\\
\end{array}
\end{equation}


\begin{equation}
\begin{array}{lll}
\frac{dS_2(t)}{S_2(t)}& =& \mu_2 (t,T) dt+\sigma_{2}(t,T)dB_{2}(t)\\
\end{array}
\end{equation}



where the volatilities $\sigma_1$ and $\sigma_2$ are positive constants and $B_1$ and $B_2$ are two Brownian motions with correlation $\rho$.

In any case, the value of an spread option $V$ at time $t$ of the spread option with date of maturity $T$ and strike $K$ is given by the following risk-neutral expectation:

\begin{equation}
\begin{array}{lll}
V(t,T)& =& \mathrm{e}^{-rT} \mathbb{E_Q}\{(S_1(T)-S_2(T)-K)^+\}\\
 & =& \mathrm{e}^{-rT} \mathbb{E_Q}\{(S_1(t)\mathrm{e}^{(\mu_1 - \sigma_1^2/2)T+\sigma_1 B_1(T) }-S_2(t)\mathrm{e}^{(\mu_2 - \sigma_2^2/2)T+\sigma_2 B_2(T) }-K)^+\}\\
 & =& \mathrm{e}^{-rT} \int \int (S_1(t)\mathrm{e}^{(\mu_1 - \sigma_1^2/2)T+\sigma_1 B_1(T) }-S_2(t)\mathrm{e}^{(\mu_2 - \sigma_2^2/2)T+\sigma_2 B_2(T) }-K)^+ \phi_T (B_1,B_2) dB_1 dB_2\\
\end{array}
\end{equation}

which shows that the value of the spread options $V(t,T)$ is given by the double integral of a function of two variables with respect to a bivariate Gaussian distribution, namely the joint distribution of $B_1(T)$ and $B_2(T)$. 


In [97]:
class SpreadOption(object):
    
    ''' Class for valuation of European Spreads Options (Calls/P.uts).
    S1_t : float : initial forward price/index level
    S2_t : float : initial forward price/index level
    K : float : strike price
    T : float : maturity (in year fractions)
    r : float : constant risk-free short rate
    vol1 : float : volatility factor in diffusion term (std)
    vol2 : float : volatility factor in diffusion term (std)
    rho : float: 
    CallPut : integer : 1 for a Call, and -1 for a Put
    '''
    
    def __init__(self, S1_t, S2_t, K, T, r, vol1, vol2, rho, CallPut):
        try: 
            self.S1_t = float(S1_t)
            self.S2_t = float(S2_t)
            self.K = float(K)
            self.T = float (T)
            self.r = float(r)
            self.vol1 = float(vol1)
            self.vol2 = float(vol2)
            self.rho = float(rho)
            self.CallPut = int(CallPut)           
            if T < 0 or r < 0 or S1_t < 0 or S2_t < 0:
                raise ValueError('Negative inputs not allowed.')
            if vol1 < 0 or vol2 < 0:
                raise ValueError('Negative volatilities are not allowed.')
            if rho > 1 or rho < -1:
                raise ValueError('Correlation out of range')
            if CallPut != 1 and CallPut != -1:
                raise ValueError('For a Call: CallPut=1, or -1 for a Put')
        except ValueError: 
            print('Error passing spread option inputs')
        


Unfortunately, except in the case of an exchange option (i.e., an option with strike K = 0), the price of the spread option cannot be given by a formula in closed form.

# Spread Options Valuation when $K=0$: The Margrabe Formula

The most common treatment of spread options is usually restricted to the special case $K = 0$ because
when the distributions of the underlying $S_1$ and $S_2$ are log-normal, this is the only case for
which one has a solution in a closed form a la `Black-Scholes` for the price of the spread option with
zero strike. This formula was first derived by Margrabe as early as in 1978, and it bears his
name:


\begin{equation}
\begin{array}{lll}
V(t,T)_{Margrabe}& =& S_2(t)\phi(d_1) - S_1(t)\phi(d_0)\\
\end{array}
\end{equation}
where 
\begin{equation}
\begin{array}{lll}
d_0 & =& \frac{\ln(S_2(t)/S_1(t))}{\sigma \sqrt{T}}-\frac{1}{2} \sigma \sqrt{T}\\
\end{array}
\end{equation}


\begin{equation}
\begin{array}{lll}
d_1 & =& \frac{\ln(S_2(t)/S_1(t))}{\sigma \sqrt{T}}+\frac{1}{2} \sigma \sqrt{T}\\
\end{array}
\end{equation}
and 
\begin{equation}
\begin{array}{lll}
\sigma & =& \sigma_1^2 + \sigma_2^2 - 2 \rho \sigma_1 \sigma_2
\end{array}
\end{equation}

In [98]:
class margrabe(SpreadOption):
    def __init__ (self, S1_t, S2_t, K, T, r, vol1, vol2, rho, CallPut):
        SpreadOption.__init__(self, S1_t, S2_t, K, T, r, vol1, vol2, rho, CallPut)
        if K != 0:
            raise ValueError('Strike should be null to use Margrabe')
            
    @property
    def price(self):
        vol = self.vol1 ** 2 + self.vol2 ** 2 - 2 * self.rho* self.vol1 * self.vol2
        d0 = (np.log(self.S2_t / self.S1_t) / (vol * np.sqrt(self.T)) - 0.5 * vol * np.sqrt(self.T))
        d1 = d0 + vol * np.sqrt(self.T)
        price = (self.CallPut * (self.S2_t  * norm.cdf(self.CallPut * d1, 0, 1) 
                                  - self.S1_t * norm.cdf(self.CallPut * d0, 0, 1)))
        return price
        

In [110]:
SpreadCALL = margrabe(40, 40, 0., .75, 0., .5, .5, .05, 1)
SpreadCALL.price

6.5183970020555684

In [111]:
def margrabe_experiment():
    for S1_t in (36., 40., 44.):  # initial stock price values
        for S2_t in (44., 40., 36.):
            print '-'*90
            for rho in (.01, .5, .99):
                SpreadCALL = margrabe(S1_t, S2_t, 0, 1, .0, 0.5, 0.5, rho, 1)
                print "Initial prices: %s, Sigmas: %s, Correlation: %2.1f --> Option Value: %3.2f" \
                         % ((S1_t, S2_t), (0.5, 0.5), rho, SpreadCALL.price)
            

In [114]:
from time import time
t0 = time()
margrabe_experiment()
t1 = time(); d1 = t1 - t0
print '-'*90
print "Duration in Seconds %6.3f" % d1

------------------------------------------------------------------------------------------
Initial prices: (36.0, 44.0), Sigmas: (0.5, 0.5), Correlation: 0.0 --> Option Value: 12.44
Initial prices: (36.0, 44.0), Sigmas: (0.5, 0.5), Correlation: 0.5 --> Option Value: 9.18
Initial prices: (36.0, 44.0), Sigmas: (0.5, 0.5), Correlation: 1.0 --> Option Value: 8.00
------------------------------------------------------------------------------------------
Initial prices: (36.0, 40.0), Sigmas: (0.5, 0.5), Correlation: 0.0 --> Option Value: 9.59
Initial prices: (36.0, 40.0), Sigmas: (0.5, 0.5), Correlation: 0.5 --> Option Value: 6.11
Initial prices: (36.0, 40.0), Sigmas: (0.5, 0.5), Correlation: 1.0 --> Option Value: 4.00
------------------------------------------------------------------------------------------
Initial prices: (36.0, 36.0), Sigmas: (0.5, 0.5), Correlation: 0.0 --> Option Value: 7.04
Initial prices: (36.0, 36.0), Sigmas: (0.5, 0.5), Correlation: 0.5 --> Option Value: 3.58
Initia